In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_community.document_loaders import UnstructuredXMLLoader

from source.ingest.preprocessing import build_documents_from_xml

In [64]:
docs = build_documents_from_xml("../data/001_상해보험_가공.xml")
docs[0]

Document(metadata={'gwan': '제1관  목적  및 용어의 정의', 'jo': '제1조(목적)', 'source': '../data/001_상해보험_가공.xml'}, page_content='이 보험계약(이하  계약 이라 합니다)은 보험계약자(이하  계약자 라 합니다)와 보험회사(이하  회사 라\n합니다) 사이에')

In [65]:
# 단계 1: 문서 로드(Load Documents)
# loader = UnstructuredXMLLoader(
#     "../data/001_상해보험_가공.xml",
#     mode="elements"   # 조항/문단 단위 분리
# )
# docs = loader.load()
# docs

docs = build_documents_from_xml("../data/001_상해보험_가공.xml")
docs[0]


Document(metadata={'gwan': '제1관  목적  및 용어의 정의', 'jo': '제1조(목적)', 'source': '../data/001_상해보험_가공.xml'}, page_content='이 보험계약(이하  계약 이라 합니다)은 보험계약자(이하  계약자 라 합니다)와 보험회사(이하  회사 라\n합니다) 사이에')

In [66]:
# # 단계 2: 문서 분할(Split Documents)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# split_documents = text_splitter.split_documents(docs)
# print(f"분할된 청크의수: {len(split_documents)}")

In [67]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

In [68]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

In [69]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [89]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
#     """당신은 보험 약관 QA 어시스턴트입니다.
# 아래 Context는 반드시 약관 원문입니다. 모르면 모른다고 답하세요.

# - 답변에는 반드시 근거 조항을 함께 제시하세요.
# - 형식: "근거: 제n관, 제n조"
# - 근거 조항은 가장 처음에 제시하세요.
# - 근거 조항이 여러 개인 경우 모두 제시하세요.
# - 근거 조항을 찾지 못한 경우, 모른다고 답하세요.

"""
너는 보험 약관 기반 판단을 수행하는 AI 상담원이다.
반드시 아래 형식으로만 답변하라.

1. 결론
2. 판단 근거 (조항 번호 포함)
3. 부족한 정보 목록
4. 지급 가능 시나리오
5. 다음 단계 안내

추측하지 말고, 약관에 근거한 내용만 사용하라.
#질문
{question}

#약관
{context}

#답변
"""
)

In [90]:
# 단계 7: 언어모델(LLM) 생성
llm = ChatUpstage(
    model="solar-1-mini-chat",
    temperature=0
)

In [91]:
def format_docs(docs):
    return "\n\n".join(
        f"[{d.metadata['gwan']} {d.metadata['jo']}]\n{d.page_content}"
        for d in docs
    )


In [92]:
# 8. Chain
chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
        }
    | prompt
    | llm
    | StrOutputParser()
)


In [93]:
# 9. 실행
question = "교통사고가 났는데 보험 보장받을 수 있어?"
response = chain.invoke(question)
print(response)

1. 결론
보험 보장 가능 여부는 약관에 명시된 조건을 충족하는지 여부에 따라 결정됩니다.

2. 판단 근거
약관 제2관 보험금의 지급 제6조 2항에 따르면, 보험기간 중 피보험자가 휴일에 대중교통 이용 중 교통재해를 직접적인 원인으로 장해분류표에서 정한 장해지급률 중 3 이상 100 이하에 해당하는 장해상태가 되었을 때 휴일 대중교통재해 장해보험금을 지급합니다.

3. 부족한 정보 목록
- 사고 발생 날짜 (휴일 여부 확인)
- 대중교통 이용 여부 확인
- 장해지급률 확인

4. 지급 가능 시나리오
- 사고 발생 날짜가 휴일일 경우
- 대중교통 이용 중 사고가 발생한 경우
- 장해지급률이 3 이상 100 이하에 해당하는 경우

5. 다음 단계 안내
- 사고 발생 날짜, 대중교통 이용 여부, 장해지급률 등의 정보를 수집하여 약관 조건을 충족하는지 확인합니다.
- 약관 조건을 충족하는 경우, 약관 제10조에 따라 필요한 서류를 준비하여 보험금 청구를 진행합니다.
- 약관 조건을 충족하지 않는 경우, 보험 보장이 불가능합니다.


In [94]:
question = "상해로 입원했을 때 보험금은 어떻게 지급되나요?"
response = chain.invoke(question)
print(response)

1. 결론
상해로 입원했을 때 보험금은 약관에 따라 지급됩니다.

2. 판단 근거
해당 약관에서는 상해로 인한 입원 시 보험금 지급에 대한 구체적인 내용이 없습니다. 다만, 보험금 지급사유에 대한 내용이 있으며, 보험기간 중 피보험자가 대중교통 이용 중 교통재해를 직접적인 원인으로 장해분류표에서 정한 장해지급률 중 3 이상 100 이하에 해당하는 장해상태가 되었을 때 보험금을 지급한다고 명시되어 있습니다.

3. 부족한 정보 목록
- 상해의 원인 및 정도
- 입원 기간
- 장해 상태 및 장해지급률

4. 지급 가능 시나리오
- 피보험자가 대중교통 이용 중 교통재해를 직접적인 원인으로 장해분류표에서 정한 장해지급률 중 3 이상 100 이하에 해당하는 장해상태가 되었을 때, 보험금을 지급받을 수 있습니다.

5. 다음 단계 안내
- 상해의 원인 및 정도, 입원 기간, 장해 상태 및 장해지급률 등의 정보를 제공하여 보험금 지급 가능 여부를 정확히 판단할 수 있도록 해주시기 바랍니다.
- 보험금 청구 서류를 준비하여 보험회사에 제출하시기 바랍니다.
- 보험금 청구 서류는 위조 또는 변조하지 않도록 주의하시기 바랍니다.
